In [50]:
import pandas as pd
import numpy as np

sale_data = pd.read_csv("EV_POP_2024_BAT.csv")
sale_data["VIN (1-10)"]

0        5YJSA1E6XR
1        YSMFD3KAXR
2        WA16AAGE9R
3        1GT10BDD2R
4        5YJ3E1EB3R
            ...    
37939    3FMTK3SU7R
37940    W1N9M0KB1R
37941    W1N9M0CB4R
37942    1FT6W3L76R
37943    KM8KRDDF4R
Name: VIN (1-10), Length: 37944, dtype: object

In [51]:
# Source 1: https://www.findmyelectric.com/tesla-vin-decoder/
# Only has data up till 2023 and does not include cybertrucks
# Source 2: https://teslatap.com/vin-decoder/
# Source 3: https://www.shop4tesla.com/en/pages/tesla-ratgeber-tesla-vin-decoder
# Lines with * at the end are not included in Source 1 and are based on Source 2 & 3

# VIN Digits 1-3: World Manufacturing Identifier
tesla_vin_1_3 = {
    "5YJ": "Fremont, California (now designated for Model S & Model 3 from 2022+)",
    "7SA": "Fremont, California (now designated for Model X & Model Y from 2022+)",
    "7G2": "Fremont, California (truck)",  # *
    "LRW": "China",
    "XP7": "Germany",
    "SFZ": "UK (Roadster 1)"
}

# VIN Digit 4: Make/Line/Series
tesla_vin_4 = {
    "S": "Tesla Model S",
    "X": "Tesla Model X",
    "3": "Tesla Model 3",
    "Y": "Tesla Model Y",
    "R": "Tesla Roadster",
    "C": "Cybertruck"  # *
}

# VIN Digit 5: Body Type and Gross Vehicle Weight Rating (GVWR)
tesla_vin_5 = {
    "A": "5 Door LHD car",
    "B": "5 Door RHD car",
    "C": "5 Door LHD Large MPV",
    "D": "5 Door RHD Large MPV",
    "E": "4 Door LHD Sedan",
    "F": "4 Door RHD Sedan",
    "G": "5 Door LHD Small MPV",
    "H": "5 Door RHD Small MPV"
}

# VIN Digit 6: Restraint System
# Cybertruck VIN digit 6 stands for a different meaning, and should not be used for interpretation in this project
tesla_vin_6 = {
    "1": "2 Front, 3 Rear Seat Belts, Front Airbags, PODS, Side Inflatable Restraints, Knee Airbags (Front)",
    "2": "TBA, pre 2014 cars",
    "3": "2 Front, 2 Rear Seat Belts, Front Airbags, Side Inflatable Restraints, Knee Airbags (Front)",
    "4": "2 Front, 3 Rear Seat Belts, Front Airbags, Side Inflatable Restraints, Knee Airbags (Front)",
    "5": "2 Front, 2 Rear Seat Belts, Front Airbags, Side Inflatable Restraints",
    "6": "2 Front, 3 Rear Seat Belts, Front Airbags, Side Inflatable Restraints",
    "7": "2 Front, 3 Rear Seat Belts, Front Airbags, Side Inflatable Restraints & Active Hood",
    "8": "2 Front, 2 Rear Seat Belts, Front Airbags, Side Inflatable Restraints & Active Hood",
    "A": "2 Front, 3 Rear, 2 3rd Row Seat Belts, Front Airbags, PODS, Side Inflatable Restraints, Knee Airbags (Front)",
    "B": "2 Front, 2 Rear, 2 3rd Row Seat Belts, Front Airbags, PODS, Side Inflatable Restraints",
    "C": "2 Front, 2 Rear, 2 3rd Row Seat Belts, Front Airbags, PODS, Side Inflatable Restraints",
    "D": "2 Front, 3 Rear Seat Belts, Front Airbags, PODS, Side Inflatable Restraints, knee airbags"
}

# VIN Digit 7: Charger Type, Battery Type, Fuel Type
# Charger Type (2012-2013) / Battery Type (2014-2015) / Fuel Type (2015+)
testa_vin_7 ={
    "A": "10kw Charger",
    "B": "20kw Charger",
    "C": "10kw Charger, with DC Fast Charge",
    "D": "20kw Charger, with DC Fast Charge",
    "E": "typically Extended Capacity Lithium Ion Battery (aka NMC or NCA) (90 kWh, P90D Ludicrous)",
    "F": "typically Lithium Ion Battery (aka LFP)",
    "H": "High Capacity Lithium Ion Battery (aka LFP) (85 kWh)",
    "S": "Standard Capacity Lithium Ion Battery (aka NMC or NCA) (60 kWh until retired and replaced with 70 kWh battery in mid 2015)",
    "V": "Very High Capacity Lithium Ion Battery (aka NMC or NCA) (90 kWh)"
}

# VIN Digit 8: Motor/Drive Unit
# Motor/Drive Unit (2012+) & Battery Type (2012-2013)
tesla_vin_8 = {
    "1": "Single Motor – Standard",
    "2": "Dual Motor – Standard (twin motors)",
    "3": "Single Performance Motor",
    "4": "Dual Motor – Performance (twin performance motor)",
    "5": "P2 Dual Motor – Long Range (2021+ refresh) (designated for Model S or Model X)",
    "6": "P2 Tri Motor – Plaid (Model S or Model X)",
    "A": "Single Motor – Standard / Performance (designated for Model 3)",
    "B": "Dual Motor – Standard (designated for Model 3)",
    "C": "Dual Motor – Performance (designated for Model 3)",
    "D": "Single Motor – Standard / Performance – standard winding (designated for Model Y)",
    "E": "Dual Motor – Standard – standard winding (designated for Model Y)",
    "F": "Dual Motor – Performance – standard winding (designated for Model Y)",
    "J": "Single Motor – hairpin winding",
    "K": "Dual Motor – hairpin winding",
    "L": "Performance Motor – hairpin winding",
    "R": "RWD V1 Motor",
    "S": "Single Motor - Standard - hairpin winding",  # *
    "T": "Dual Motor - Performance"  # *
}

# VIN Digit 9: Check Digit
# Ignored here

# VIN Digit 10: Model Year
testa_vin_10 = {
    "C": "2012",
    "D": "2013",
    "E": "2014",
    "F": "2015",
    "G": "2016",
    "H": "2017",
    "J": "2018",
    "K": "2019",
    "L": "2020",
    "M": "2021",
    "N": "2022",
    "P": "2023",
    "R": "2024",  # *
    "S": "2025"   # *
}

In [52]:
df_1_3 = pd.DataFrame([{"vin_1_3": k, "manufacturer": v} for k, v in tesla_vin_1_3.items()])
df_4 = pd.DataFrame([{"vin_4": k, "make": v} for k, v in tesla_vin_4.items()])
df_5 = pd.DataFrame([{"vin_5": k, "body type": v} for k, v in tesla_vin_5.items()])
df_6 = pd.DataFrame([{"vin_6": k, "restraint system": v} for k, v in tesla_vin_6.items()])
df_7 = pd.DataFrame([{"vin_7": k, "fuel type": v} for k, v in testa_vin_7.items()])
df_8 = pd.DataFrame([{"vin_8": k, "motor/drive unit": v} for k, v in tesla_vin_8.items()])
df_10 = pd.DataFrame([{"vin_10": k, "model year": v} for k, v in testa_vin_10.items()])

In [53]:
useful_columns = [
    "VIN (1-10)",
    "Model Year",
    "Make",
    "Model",
    "Primary Use",
    "Odometer Reading",
    "Odometer Reading Description",
    "New or Used Vehicle",
    "Sale Price",
    "Sale Date",
    "Transaction Type",
    "Transaction Date",
    "Year",
    "County",
    "City",
    "State",
    "Postal Code",
]
tesla_sale_data = sale_data[sale_data["Make"] == "TESLA"][useful_columns].copy()

# Split VIN parts directly in tesla_sale_data
tesla_sale_data["vin_1_3"] = tesla_sale_data["VIN (1-10)"].str[0:3]
tesla_sale_data["vin_4"]   = tesla_sale_data["VIN (1-10)"].str[3]
tesla_sale_data["vin_5"]   = tesla_sale_data["VIN (1-10)"].str[4]
tesla_sale_data["vin_6"]   = tesla_sale_data["VIN (1-10)"].str[5]
tesla_sale_data["vin_7"]   = tesla_sale_data["VIN (1-10)"].str[6]
tesla_sale_data["vin_8"]   = tesla_sale_data["VIN (1-10)"].str[7]
tesla_sale_data["vin_10"]  = tesla_sale_data["VIN (1-10)"].str[9]

# Merge each decoded part individually
tesla_sale_data = tesla_sale_data.merge(df_1_3, on="vin_1_3", how="left")
tesla_sale_data = tesla_sale_data.merge(df_4, on="vin_4", how="left")
tesla_sale_data = tesla_sale_data.merge(df_5, on="vin_5", how="left")
tesla_sale_data = tesla_sale_data.merge(df_6, on="vin_6", how="left")
tesla_sale_data = tesla_sale_data.merge(df_7, on="vin_7", how="left")
tesla_sale_data = tesla_sale_data.merge(df_8, on="vin_8", how="left")
tesla_sale_data = tesla_sale_data.merge(df_10, on="vin_10", how="left")

In [54]:
tesla_sale_data = tesla_sale_data[tesla_sale_data["make"] != "Cybertruck"]  # Removed because Cybertruck vin_6 seems to have a different meaning
tesla_sale_data = tesla_sale_data.dropna()
tesla_sale_data

,VIN (1-10),Model Year,Make,Model,Primary Use,Odometer Reading,Odometer Reading Description,New or Used Vehicle,Sale Price,Sale Date,...,vin_7,vin_8,vin_10,manufacturer,make,body type,restraint system,fuel type,motor/drive unit,model year
0,5YJSA1E6XR,2024,TESLA,Model S,Passenger,15,Actual Mileage,New,96130,June 08 2024,...,E,6,R,"Fremont, California (now designated for Model ...",Tesla Model S,5 Door LHD car,"2 Front, 3 Rear Seat Belts, Front Airbags, POD...",typically Extended Capacity Lithium Ion Batter...,P2 Tri Motor – Plaid (Model S or Model X),2024
1,5YJ3E1EB3R,2024,TESLA,Model 3,Passenger,15,Actual Mileage,New,51130,September 06 2024,...,E,B,R,"Fremont, California (now designated for Model ...",Tesla Model 3,4 Door LHD Sedan,"2 Front, 3 Rear Seat Belts, Front Airbags, POD...",typically Extended Capacity Lithium Ion Batter...,Dual Motor – Standard (designated for Model 3),2024
2,5YJ3E1EB3R,2024,TESLA,Model 3,Passenger,15,Actual Mileage,New,59880,April 24 2024,...,E,B,R,"Fremont, California (now designated for Model ...",Tesla Model 3,4 Door LHD Sedan,"2 Front, 3 Rear Seat Belts, Front Airbags, POD...",typically Extended Capacity Lithium Ion Batter...,Dual Motor – Standard (designated for Model 3),2024
4,5YJ3E1ET0R,2024,TESLA,Model 3,Passenger,15,Actual Mileage,New,54630,June 07 2024,...,E,T,R,"Fremont, California (now designated for Model ...",Tesla Model 3,4 Door LHD Sedan,"2 Front, 3 Rear Seat Belts, Front Airbags, POD...",typically Extended Capacity Lithium Ion Batter...,Dual Motor - Performance,2024
5,7SAYGDEE9R,2024,TESLA,Model Y,Passenger,15,Actual Mileage,New,49630,September 05 2024,...,E,E,R,"Fremont, California (now designated for Model ...",Tesla Model Y,5 Door LHD Small MPV,"2 Front, 3 Rear Seat Belts, Front Airbags, POD...",typically Extended Capacity Lithium Ion Batter...,Dual Motor – Standard – standard winding (desi...,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17437,7SAYGDEE3R,2024,TESLA,Model Y,Passenger,15,Actual Mileage,New,53630,September 24 2024,...,E,E,R,"Fremont, California (now designated for Model ...",Tesla Model Y,5 Door LHD Small MPV,"2 Front, 3 Rear Seat Belts, Front Airbags, POD...",typically Extended Capacity Lithium Ion Batter...,Dual Motor – Standard – standard winding (desi...,2024
17438,7SAYGDEE5R,2024,TESLA,Model Y,Passenger,15,Actual Mileage,New,46690,October 17 2024,...,E,E,R,"Fremont, California (now designated for Model ...",Tesla Model Y,5 Door LHD Small MPV,"2 Front, 3 Rear Seat Belts, Front Airbags, POD...",typically Extended Capacity Lithium Ion Batter...,Dual Motor – Standard – standard winding (desi...,2024
17439,5YJ3E1ET8R,2024,TESLA,Model 3,Passenger,15,Actual Mileage,New,58130,October 30 2024,...,E,T,R,"Fremont, California (now designated for Model ...",Tesla Model 3,4 Door LHD Sedan,"2 Front, 3 Rear Seat Belts, Front Airbags, POD...",typically Extended Capacity Lithium Ion Batter...,Dual Motor - Performance,2024
17440,7SAXCBE68R,2024,TESLA,Model X,Passenger,15,Actual Mileage,New,100130,November 07 2024,...,E,6,R,"Fremont, California (now designated for Model ...",Tesla Model X,5 Door LHD Large MPV,"2 Front, 2 Rear, 2 3rd Row Seat Belts, Front A...",typically Extended Capacity Lithium Ion Batter...,P2 Tri Motor – Plaid (Model S or Model X),2024


# Descriptive Analysis
Provide a comprehensive summary of your combined dataset using descriptive statistics. This should include means, medians, modes, ranges, variance, and standard deviations for the relevant features of your data.  The descriptive statistics should inform your guiding questions that you developed in Part I of the project, rather than merely providing an overview of your data.  Interpret these results to draw preliminary conclusions about the data.

In [60]:
# Seems to be the only columns that make sense to compute these statistics
tesla_sale_data[["Sale Price", "Odometer Reading", "Year"]].describe()

,Sale Price,Odometer Reading,Year
count,16258.000000,16258.000000,16258.000000
mean,52080.674929,50.787243,2024.010395
std,12145.520002,760.233485,0.101427
min,1630.000000,1.000000,2024.000000
25%,47630.000000,15.000000,2024.000000
50%,50630.000000,15.000000,2024.000000
75%,52630.000000,15.000000,2024.000000
max,522630.000000,93024.000000,2025.000000


# Inferential Statistics
Conduct appropriate hypothesis tests to investigate if there are significant differences or correlations within your data.  This might involve regression analysis, ANOVA, and/or chi-squared tests.

Clearly state your null and alternative hypotheses, choose an appropriate significance level, and discuss your findings. Make sure to justify the choice of your tests.


# Graphical Analysis
Create various types of plots to visualize relationships within your data. Use histograms, bar charts, scatter plots, box plots, and any other suitable graphical representations you've learned.

Be sure to use appropriate titles, labels, and legends to make your plots readable and informative.

Interpret the graphical representations to uncover patterns, trends, and outliers.


# Comparative Analysis
Compare and contrast different subsets of your data. This can include comparisons over time, across different categories, or any other relevant segmentation.  Note that for some projects, the nature of this comparative analysis will be obvious.  For others, you will need to think about how you might subset your data.

Discuss any notable similarities or differences you have identified.


# Multivariate Analysis
Perform multivariate analysis to understand the relationships among three or more variables in your dataset.

Use techniques like cross-tabulation, pivot tables, and multivariate graphs.


# Synthesis
Synthesize the findings from your descriptive and inferential statistics along with your graphical analyses to answer your research questions.

Discuss how the combination of the datasets has provided added value in terms of insights or capabilities that would not be possible with the individual datasets in isolation.


# Documentation
Outline the steps you took in your analysis, providing the rationale for the choice of tools and techniques.

Clearly document your code, analyses, and interpretations so that they are understandable and reproducible.


# Reporting & Interpretation
Prepare markdown blocks that summarize your methodology, analysis, and findings.  You should intersperse these with your code.

Your notebook should be well-structured, with clear sections, headings, and subheadings.


# Reflection
Include a section (using one or more markdown blocks) at the end of your notebook in which you reflect on the process of analyzing the data. Discuss any challenges you encountered and how you overcame them.

Critically evaluate the limitations of your analysis and suggest areas for further research or improvement.
